In [1]:
import qiskit
from qiskit_aer.primitives import SamplerV2
import re


In [2]:
noncomment_re = re.compile("^([^#]*)(#.*)?$")
vars_re = re.compile("^\\.[vV] (.*)$")
inputs_re = re.compile("^\\.[iI] (.*)$")
outputs_re = re.compile("^\\.[oO] (.*)$")
begin_re = re.compile("^BEGIN\\s?.*$")
end_re = re.compile("^END\\s?.*$")
gate_re = re.compile("(\\S+)\\s+(.+)")
decl_re = re.compile("[A-Za-z][0-9A-Za-z]*")

split_ws_re = re.compile("\\s+")

#circ.measure_all()    for line_no, l, raw_line in lines:

def H_gate(circ, _, ctl, tgt):
    if len(ctl) > 0:
        return None
    return circ.h(tgt)

def S_gate(circ, _, ctl, tgt):
    if len(ctl) > 0:
        return None
    return circ.s(tgt)

def Sdg_gate(circ, _, ctl, tgt):
    if len(ctl) > 0:
        return None
    return circ.sdg(tgt)

def T_gate(circ, _, ctl, tgt):
    if len(ctl) > 0:
        return None
    return circ.t(tgt)

def Tdg_gate(circ, _, ctl, tgt):
    if len(ctl) > 0:
        return None
    return circ.tdg(tgt)

def X_gate(circ, _, ctl, tgt):
    if len(ctl) == 0:
        return circ.x(tgt)
    elif len(ctl) == 1:
        return circ.cx(ctl[0], tgt)
    elif len(ctl) == 2:
        return circ.ccx(ctl[0], ctl[1], tgt)
    else:
        return circ.mcx(ctl, tgt)

def Y_gate(circ, _, ctl, tgt):
    if len(ctl) > 0:
        return None
    return circ.y(tgt)

def Z_gate(circ, _, ctl, tgt):
    if len(ctl) == 0:
        return circ.z(tgt)
    elif len(ctl) == 1:
        return circ.cz(ctl[0], tgt)
    elif len(ctl) == 2:
        return circ.ccz(ctl[0], ctl[1], tgt)
    else:
        return None

def Zd_gate(circ, _, ctl, tgt):
    if len(ctl) != 2:
        return None
    circ.tdg(ctl[0])
    circ.tdg(ctl[1])
    circ.cx(tgt, ctl[0])
    circ.t(ctl[0])
    circ.cx(ctl[1], tgt)
    circ.t(tgt)
    circ.cx(ctl[1], ctl[0])
    circ.tdg(ctl[0])
    circ.cx(ctl[1], tgt)
    circ.cx(tgt, ctl[0])
    circ.t(ctl[0])
    circ.tdg(tgt)
    return circ.cx(ctl[1], ctl[0])

def swap_gate(circ, _, ctl, tgt):
    if len(ctl) != 1:
        return None
    return circ.swap(ctl[0], tgt)

accepted_gates = {
    "H": H_gate,
    "P": S_gate,
    "P*": Sdg_gate,
    "S": S_gate,
    "S*": Sdg_gate,
    "T": T_gate,
    "T*": Tdg_gate,
    "X": X_gate,
    "cnot": X_gate,
    "tof": X_gate,
    "Y": Y_gate,
    "Z": Z_gate,
    "tof": X_gate,
    "cnot": X_gate,
    "Z": Z_gate,
    "cz": Z_gate,
    "Zd": Zd_gate,
    "swap": swap_gate,
}
#        ("swap", [x,y]) -> [Swap x y]

def parse_qc(qc_src):
    lines = [(n, m.group(1).strip(), l) for n, m, l in
                 ((n, noncomment_re.match(l), l) for n, l in
                     enumerate(qc_src.split("\n")))
                 if m and m.group(1).strip() != ""]
    var_decls = None
    input_decls = None
    output_decls = None
    all_decls = None
    decl_ids = {}
    circ = None
    state = "decls"
    for line_no, l, raw_line in lines:
        if state == "decls":
            if begin_re.match(l):
                all_decls = set()
                if var_decls is not None:
                    all_decls.update(var_decls)
                if input_decls is not None:
                    all_decls.update(input_decls)
                if output_decls is not None:
                    all_decls.update(output_decls)
                if not all_decls:
                    print(f"Invalid .qc: {line_no}: no variables declared")
                    return None
                for decl in all_decls:
                    if not decl_re.match(decl):
                        print(f"Invalid .qc: {line_no}: bad decl name '{decl}'")
                        return None
                    decl_ids[decl] = len(decl_ids)
                circ = qiskit.QuantumCircuit(len(decl_ids))
                state = "gates"
                continue
            if vars_re.match(l):
                if var_decls is not None:
                    print(f"Invalid .qc: {line_no}: multiple .v lines")
                    return None
                var_decls = split_ws_re.split(l)[1:]
                continue
            if inputs_re.match(l):
                if input_decls is not None:
                    print(f"Invalid .qc: {line_no}: multiple .i lines")
                    return None
                input_decls = split_ws_re.split(l)[1:]
                continue
            if outputs_re.match(l):
                if output_decls is not None:
                    print(f"Invalid .qc: {line_no}: multiple .o lines")
                    return None
                output_decls = split_ws_re.split(l)[1:]
                continue
            print(f"Invalid .qc: {line_no}: expecting declarations or BEGIN, couldn't parse '{raw_line}'")
            return None
        elif state == "gates":
            if end_re.match(l):
                state = "end"
                continue
            m = gate_re.match(l)
            if not m:
                print(f"Invalid .qc: {line_no}: expecting gate, couldn't parse '{raw_line}'")
                return None
            gate_name = m.group(1)
            if gate_name not in accepted_gates:
                print(f"Invalid .qc: {line_no}: unrecognized gate '{gate_name}'")
                return None
            if not m.group(2):
                print(f"Invalid .qc: {line_no}: no target for gate '{gate_name}'")
                return None
            gate_refs = split_ws_re.split(m.group(2))
            ref_ids = []
            for r in gate_refs:
                if r not in all_decls:
                    print(f"Invalid .qc: {line_no}: '{gate_name}' references undeclared qubit '{r}'")
                    return None
                ref_ids.append(decl_ids[r])
            accepted_gates[gate_name](circ, gate_name, ref_ids[:-1], ref_ids[-1])
        elif state == "end":
            print(f"Invalid .qc: {line_no}: statements after end")
        else:
            print("wtf")
            return None
    if state != "end":
        print(f"Invalid .qc: {line_no + 1}: expecting END, source truncated")
    return circ

    


In [3]:
# Generate 3-qubit GHZ state
def simulate(qc_src):
    circ = parse_qc(qc_src)
    if circ is None:
        return
    print(circ)
    circ.measure_all()
    
    # Construct an ideal simulator with SamplerV2
    sampler = SamplerV2()
    job = sampler.run([circ], shots=1024)

    # Perform an ideal simulation
    result_ideal = job.result()
    counts_ideal = result_ideal[0].data.meas.get_counts()
    print('Counts(ideal):', counts_ideal)


In [4]:
simulate("""
.v x0 x1 x2 y0 y1 y2 y3 y4 y5
.i x0 x1 x2
.o x0 x1 x2 y0 y1 y2 y3 y4 y5

BEGIN

H x0
H x1
H x2

X x2
H y4
Z x0 x2 y4
H y4
X x2

X x0
H y3
H y5
Z x1 y4 y3
Z x0 x2 y5
H y5
H y3

tof y3 y2
tof y3 y0
tof y5 y4

X x1
X y4
H y0
H y3
Z x1 y5 y3
Zd x1 y4 y0
H y3
H y0
X x1

tof y3 y1
tof y2 y5

H y2
H y5
Zd x1 y4 y2
Zd x0 x2 y5
H y5
H y2
X x0
X y4

tof y2 y5

END
""")


     ┌───┐                 ┌───┐                                      ┌───┐»
q_0: ┤ H ├──────────────■──┤ H ├────────■────────────────────■────────┤ X ├»
     ├───┤┌───┐   ┌───┐ │ ┌┴───┴┐       │                    │        └─┬─┘»
q_1: ┤ H ├┤ X ├─■─┤ X ├─■─┤ Tdg ├───────┼────────────────────┼──────────┼──»
     ├───┤└───┘ │ ├───┤ │ ├─────┤       │                    │          │  »
q_2: ┤ H ├──────■─┤ X ├─■─┤ Tdg ├───────┼────────────────────┼──────────┼──»
     ├───┤      │ └───┘   └┬───┬┘       │       ┌───┐        │  ┌───┐   │  »
q_3: ┤ H ├──────┼───────■──┤ H ├───■────┼────■──┤ H ├────────■──┤ H ├───┼──»
     ├───┤      │ ┌───┐ │  └───┘   │  ┌─┴─┐  │  ├───┤┌─────┐ │  └───┘   │  »
q_4: ┤ H ├──────■─┤ H ├─■──────────┼──┤ X ├──┼──┤ X ├┤ Tdg ├─┼──────────┼──»
     └───┘        └───┘ │        ┌─┴─┐└───┘  │  └───┘└─────┘ │          │  »
q_5: ───────────────────┼────────┤ X ├───────┼───────────────┼──────────■──»
     ┌───┐              │  ┌───┐ └───┘       │               │ ┌─────┐┌───┐»

In [5]:
simulate("""
.v x0 x1 x2 y0 y1 y2 y3 y4 y5
.i x0 x1 x2
.o x0 x1 x2 y0 y1 y2 y3 y4 y5

BEGIN

H x0
H x1
H x2

X y0
cnot x2 y0
cnot x2 y1
cnot x2 y3
cnot x2 y4
cnot y0 y4
cnot x0 y0
cnot y1 y0
cnot y1 y3
cnot y1 y5
cnot y5 y1
cnot y5 y0
cnot y2 y1
cnot y2 y4
cnot x1 y2
cnot x1 y5
tof x2 x0 y5
tof x2 x0 x1 y2
tof x2 x1 y2
tof x2 x0 x1 y0
tof x0 x1 y0
tof x2 x0 x1 y1
tof x0 x1 y1
tof y2 y5

END
""")


                                                  ┌───┐                    »
q_0: ─────────────────────────────────────────────┤ X ├──■────■────────────»
     ┌───┐                                        └─┬─┘  │    │            »
q_1: ┤ H ├──■────■────■────■────────────────────────┼────┼────┼────────────»
     ├───┤  │    │    │    │                        │    │    │            »
q_2: ┤ H ├──┼────┼────┼────┼─────────■──────────────┼────┼────┼────────────»
     └───┘  │    │  ┌─┴─┐  │         │       ┌───┐  │    │    │            »
q_3: ───────┼────┼──┤ X ├──┼─────────┼───────┤ X ├──┼────┼────┼────────────»
            │    │  └───┘┌─┴─┐┌───┐  │       └─┬─┘  │    │    │       ┌───┐»
q_4: ───────┼────┼───────┤ X ├┤ X ├──┼─────────┼────┼────┼────┼───────┤ X ├»
            │    │       └───┘└─┬─┘  │         │    │    │    │       └─┬─┘»
q_5: ───────┼────┼──────────────┼────┼─────────┼────┼────┼────┼────■────■──»
     ┌───┐  │    │              │    │         │    │    │    │    │       »

In [6]:
simulate("""
# Feynman -- quantum circuit toolkit
# Original (mod_mult_55):
#   Qubits: 9
#   H: 14
#   T: 49
#   X: 8
#   cnot: 48
#   Depth: 43
#   T depth: 19
# Result (114.882ms):
#   Qubits: 26
#   H: 20
#   S: 16
#   T: 33
#   X: 1
#   cnot: 131
#   swap: 9
#   Depth: 120
#   T depth: 27
.v A1X13 A1X14 A1X15 A1X16 A1X17 A1X19 A1X20 A1X21 A1X22 A1X23 A1X24 A1Xi25 A1Xi27 A1Xi28 A1Xi30 A1Xi33 x0 x1 x2 y0 y1 y2 y3 y4 y5 anc0
.i x0 x1 x2
.o x0 x1 x2 y0 y1 y2 y3 y4 y5
BEGIN
H x0
H x1
H x2

cnot x2 A1Xi25
cnot x0 A1Xi27
cnot y3 A1Xi28
cnot x1 A1Xi30
cnot y1 A1Xi33
cnot y4 A1X13
cnot x1 A1X13
H A1X14
S* A1X13
cnot A1X14 y4
cnot A1X13 A1X14
cnot A1X13 y4
cnot A1X13 A1X14
cnot A1X14 y4
cnot A1X13 y4
H A1X14
H A1X15
cnot A1X15 y5
cnot x1 A1X15
cnot x1 y5
cnot x1 A1X15
cnot A1X15 y5
cnot x1 y5
H A1X15
cnot y3 A1X16
cnot y0 A1X16
cnot A1X15 A1X17
cnot A1X16 A1X17
cnot A1X14 A1X19
cnot A1X16 A1X19
cnot y4 A1X20
cnot y3 A1X20
cnot y5 A1X21
cnot A1X20 A1X21
cnot y0 A1X22
cnot A1X21 A1X22
cnot A1X14 A1X23
cnot A1X22 A1X23
cnot y2 A1X24
cnot y0 A1X24
H A1X23
T A1Xi27
cnot A1X23 A1Xi27
T* A1Xi27
cnot A1Xi30 A1X23
S* A1X23
cnot A1Xi30 A1Xi27
T A1Xi27
cnot A1Xi30 A1X23
cnot A1X23 A1Xi27
S* A1Xi27
S A1X23
cnot A1Xi30 A1Xi27
H anc0
cnot anc0 A1Xi25
T* A1Xi25
cnot A1Xi27 anc0
T* anc0
cnot A1Xi27 A1Xi25
T A1Xi25
cnot A1Xi27 anc0
cnot anc0 A1Xi25
T A1Xi25
T anc0
cnot A1Xi27 A1Xi25
H anc0
T anc0
cnot A1X23 anc0
T* anc0
cnot A1Xi30 A1X23
cnot A1Xi30 anc0
T anc0
cnot A1Xi30 A1X23
cnot A1X23 anc0
T* anc0
cnot A1Xi30 anc0
H A1X23
H anc0
cnot anc0 A1Xi25
S* A1Xi25
cnot A1Xi27 anc0
S* anc0
cnot A1Xi27 A1Xi25
S A1Xi25
cnot A1Xi27 anc0
cnot anc0 A1Xi25
S anc0
cnot A1Xi27 A1Xi25
H A1X19
cnot A1X19 A1Xi27
T* A1Xi27
cnot A1Xi30 A1X19
S* A1X19
cnot A1Xi30 A1Xi27
T A1Xi27
cnot A1Xi30 A1X19
cnot A1X19 A1Xi27
S A1X19
cnot A1Xi30 A1Xi27
cnot anc0 A1Xi25
cnot A1Xi27 anc0
cnot A1Xi27 A1Xi25
cnot A1Xi27 anc0
cnot anc0 A1Xi25
cnot A1Xi27 A1Xi25
H anc0
T anc0
cnot A1X19 anc0
T* anc0
cnot A1Xi30 A1X19
cnot A1Xi30 anc0
T anc0
cnot A1Xi30 A1X19
cnot A1X19 anc0
T* anc0
cnot A1Xi30 anc0
H A1X19
H anc0
cnot anc0 A1Xi25
S* A1Xi25
cnot A1Xi27 anc0
S* anc0
cnot A1Xi27 A1Xi25
S A1Xi25
cnot A1Xi27 anc0
cnot anc0 A1Xi25
S anc0
cnot A1Xi27 A1Xi25
H A1X17
cnot A1X17 A1Xi25
T* A1Xi25
cnot A1Xi30 A1X17
S* A1X17
cnot A1Xi30 A1Xi25
T A1Xi25
cnot A1Xi30 A1X17
cnot A1X17 A1Xi25
T* A1Xi25
S A1X17
cnot A1Xi30 A1Xi25
cnot anc0 A1Xi25
cnot A1Xi27 anc0
cnot A1Xi27 A1Xi25
cnot A1Xi27 anc0
cnot anc0 A1Xi25
cnot A1Xi27 A1Xi25
H anc0
T anc0
cnot A1X17 anc0
T* anc0
cnot A1Xi30 A1X17
cnot A1Xi30 anc0
T anc0
cnot A1Xi30 A1X17
cnot A1X17 anc0
T* anc0
cnot A1Xi30 anc0
H A1X17
H anc0
cnot anc0 A1Xi25
T* A1Xi25
cnot A1Xi27 anc0
T* anc0
cnot A1Xi27 A1Xi25
T A1Xi25
cnot A1Xi27 anc0
cnot anc0 A1Xi25
T anc0
cnot A1Xi27 A1Xi25
H anc0
H A1Xi28
cnot A1Xi28 A1Xi25
T* A1Xi25
cnot A1Xi27 A1Xi28
T* A1Xi28
cnot A1Xi27 A1Xi25
T A1Xi25
cnot A1Xi27 A1Xi28
cnot A1Xi28 A1Xi25
T A1Xi28
cnot A1Xi27 A1Xi25
H A1Xi28
cnot A1Xi33 A1Xi28
cnot A1X24 A1X23
cnot A1X24 A1X17
cnot A1Xi30 A1Xi28
cnot A1Xi30 A1X17
cnot A1X17 A1X24
cnot A1X17 A1X23
cnot A1Xi28 A1X19
cnot A1Xi28 A1X23
cnot A1X23 A1Xi28
cnot A1X23 A1Xi33
cnot A1X23 A1X19
cnot A1Xi27 A1X19
cnot A1X19 A1X24
cnot A1Xi25 A1X24
cnot A1Xi25 A1Xi33
cnot A1Xi25 A1X23
cnot A1Xi25 A1X19
X A1X19
swap y5 A1Xi28
swap y4 A1X24
swap y3 A1Xi33
swap y2 A1X17
swap y1 A1X23
swap y0 A1X19
swap x2 A1Xi25
swap x1 A1Xi30
swap x0 A1Xi27
END
""")

                ┌───┐                                                         »
 q_0: ──────────┤ X ├─────────────────────────────────────────────────────────»
                └─┬─┘                                                         »
 q_1: ────────────┼───────────────────────────────────────────────────────────»
      ┌───┐       │                                                           »
 q_2: ┤ H ├───────┼─────────■─────────────────────────────────────────────────»
      ├───┤       │         │                                                 »
 q_3: ┤ H ├───────┼─────────┼────■────────────────────────────────────────────»
      └───┘       │  ┌───┐  │    │                                            »
 q_4: ────────────┼──┤ H ├──┼────┼────────────────────────────────────■───────»
                  │  └───┘  │    │       ┌───┐                        │       »
 q_5: ──■─────────┼─────────┼────┼───────┤ X ├────────────────────────┼───────»
        │         │  ┌───┐  │    │      